# Manufacturing Efficiency Estimation Calculations

This notebook covers the calculations used to estimate the manufacturing efficiency of silicon for mono and mc Si modules. This data will be utilized in the baseline files for PV DEMICE.

NOTE: You must run Silicon per m2 journal before running this one, as the calculations depend on grams of silicon per cell.

In [2]:
import numpy as np
import pandas as pd
import os,sys
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 22})
plt.rcParams['figure.figsize'] = (12, 8)

In [5]:
#read in supporting csv files
cwd = os.getcwd() #grabs current working directory
si_g_percell = pd.read_csv(cwd+"/../../PV_DEMICE/baselines/SupportingMaterial/si_g_per_cell.csv", index_col='Year')
kerf_loss_raw = pd.read_csv(cwd+"/../../PV_DEMICE/baselines/SupportingMaterial/kerf_loss_microns.csv", index_col='Year')
utilize_gperwafer_raw = pd.read_csv(cwd+"/../../PV_DEMICE/baselines/SupportingMaterial/utilization_g_perwafer.csv", index_col='Year')
wafer_thickness = pd.read_csv(cwd+"/../../PV_DEMICE/baselines/SupportingMaterial/Wafer_thickness.csv", index_col='Year')


From the literature resources, we have data points that indicate a 50% input material waste for Silicon per cell prior to 2003. Therefore, we will assume a 50% manufacturing efficiency 1995 through 2003.

There is a derth of data 2004 through 2009.

As of 2010, ITRPV data provides kerf loss data, through 2017. This combined with wafer thickness, will be used as a proxy for mfg losses, combined with insights from the ITRPV "polysilicon utilization" data (which includes wafer thickness, kerf loss, crucible size, from squaring to cropping).

Similarly, as of 2017, ITRPV provides data on "polysilicon utilization", which is expressed as a number of grams of silicon used to make a wafer, and proken down by mono-Si vs. mc-Si and diamond wire vs. slurry cut, and includes projections out to 2030. This data will be utilized for 2017 onward to calculate % manufacturing efficiency relative to the average cell mass calculated by "Silicon g per m2" OR shold we use on a 10g/cell basis that ITRPV uses??

## 2010 through 2017

mfg efficiency by kerf loss proxy is calculated by:
            
            [Wafer Thickness/(Wafer Thickness + Kerf Loss)] * 100 = % mfg efficiency (proxy)

In [14]:
#There are missing data in wafer thickness, so we will interpolate linearly for missing years
wafer_thick = wafer_thickness.interpolate(method='linear',axis=0)

#Kerf loss also has missing data (projections), same linear interpolate
kerf_loss = kerf_loss_raw.interpolate(method='linear',axis=0, limit_area='inside')

#concat wafer thickness and kerf loss dfs together
df_thick_kerf = pd.concat([wafer_thick,kerf_loss], axis=1) #concatinate on the columns axis

#print(df_thick_kerf)

Because slurry and diamond have significantly different kerf losses, we will keep these seperate for as long as possible before averaging.

In [17]:
df_thick_kerf['slurry_eff'] = df_thick_kerf['wafer_thickness']/(df_thick_kerf['wafer_thickness']+df_thick_kerf['slurry'])
df_thick_kerf['diamond_eff'] = df_thick_kerf['wafer_thickness']/(df_thick_kerf['wafer_thickness']+df_thick_kerf['diamond'])

print(df_thick_kerf)

      wafer_thickness  slurry     diamond  slurry_eff  diamond_eff
Year                                                              
1990       400.000000     NaN         NaN         NaN          NaN
1991       392.857143     NaN         NaN         NaN          NaN
1992       385.714286     NaN         NaN         NaN          NaN
1993       378.571429     NaN         NaN         NaN          NaN
1994       371.428571     NaN         NaN         NaN          NaN
1995       364.285714     NaN         NaN         NaN          NaN
1996       357.142857     NaN         NaN         NaN          NaN
1997       350.000000     NaN         NaN         NaN          NaN
1998       342.857143     NaN         NaN         NaN          NaN
1999       335.714286     NaN         NaN         NaN          NaN
2000       328.571429     NaN         NaN         NaN          NaN
2001       321.428571     NaN         NaN         NaN          NaN
2002       314.285714     NaN         NaN         NaN         

## 2017 through 2030

## 1995 through 2003

Here, as previously stated, we set 2003 and earlier years to 50% mfg efficiency

## 2004 through 2009

Until better data is found for this time period, simple linear interpolation will be used.